In [2]:
import quandl
import pandas_datareader as pdr
import investpy 
import yfinance
import datetime
from hmmlearn import hmm
from finpie.economic_data import oecd_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%run keys.ipynb
# from sklearn.preprocessing import LabelEncoder

# Dickey-Fuller Test for Stationarity
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Display all columns in the DataFrame
pd.set_option("display.max_columns", None)

### Gathering data from APIs

In [9]:
# Pulling ISM-Purchasing Managers Index data from Quandl API
ism_df = pd.read_csv('https://www.quandl.com/api/v3/datasets/ISM/MAN_PMI.csv?' + 'api_key=' + quandl.ApiConfig.api_key, 
                     index_col=['Date'])
# Flipping the DataFrame
ism_df = ism_df.iloc[::-1]
# Cleaning the DataFrame
ism_df.columns = ['pmi']
#DateTime Format
ism_df.index = pd.to_datetime(ism_df.index)


# Pulling data from FRED 
data_source = "fred"
start = datetime.datetime (1960, 1, 1)
end = datetime.datetime (2030, 12, 1)

indicators = pdr.DataReader(['CPIAUCSL', 
                             'CURRCIR',
                             'FEDFUNDS',
                             'PCEPI',
                             # Manufacturing
                             'WPUSI019011',
                             'WPU101',
                             'PCU325325',
                             'WPS112'], 
                             data_source=data_source, 
                             start=start, 
                             end=end)
indicators.index.name = "Date"
indicators.columns = ['headcpi', 'm2', 'fedFunds', 'pce', 
                      'copProd', 'ironSteel', 'chemMan', 'conMachEq']

# Pulling Commodities from Investpy
gold = investpy.commodities.get_commodity_historical_data('Gold', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
gold.index.names = ['Date']
silver = investpy.commodities.get_commodity_historical_data('Silver', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
silver.index.names = ['Date']
copper = investpy.commodities.get_commodity_historical_data('Copper', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country='united states', 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
lumber = investpy.commodities.get_commodity_historical_data('Lumber', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
lumber.index.names = ['Date']
nat_gas = investpy.commodities.get_commodity_historical_data('Natural Gas', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
nat_gas.index.names = ['Date']
crude = investpy.commodities.get_commodity_historical_data('Crude Oil WTI', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
crude.index.names = ['Date']
corn = investpy.commodities.get_commodity_historical_data('US Corn', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
corn.index.names = ['Date']
wheat = investpy.commodities.get_commodity_historical_data('US Wheat', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
wheat.index.names = ['Date']
soy = investpy.commodities.get_commodity_historical_data('US Soybeans', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
soy.index.names = ['Date']
sugar = investpy.commodities.get_commodity_historical_data('US Sugar #11', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
sugar.index.names = ['Date']
coffee = investpy.commodities.get_commodity_historical_data('US Coffee C', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')
coffee.index.names = ['Date']

us10 = investpy.bonds.get_bond_historical_data(bond='U.S. 10Y', 
                                        from_date = '01/01/1980', 
                                        to_date = '01/01/2030', 
#                                         country=united states, 
                                        order='ascending', 
                                        interval='Monthly')
us10.index.names = ['Date']

dxy = investpy.indices.get_index_historical_data('US Dollar Index', 
                                        from_date = '01/01/1980', 
                                        to_date = '01/01/2030', 
                                        country='united states', 
                                        order='ascending', 
                                        interval='Monthly')
dxy.index.names = ['Date']

# TRCCRB
crb = investpy.indices.get_index_historical_data('TR/CC CRB', 
                                                 from_date = '01/01/1980', 
                                                 to_date = '01/01/2030', 
                                                 country='world', 
                                                 order='ascending', 
                                                 interval='Monthly')
crb.index.names = ['Date']                                        
                                  

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
1994-02-01,120.80,121.32,116.75,118.03,0,USD
1994-03-01,118.28,120.49,116.72,118.45,0,USD
1994-04-01,120.30,122.50,119.12,122.47,0,USD
1994-05-01,122.83,131.64,122.36,130.67,0,USD
1994-06-01,129.51,137.20,127.74,135.20,0,USD
